In [65]:
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, zero_one_loss
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sklearn.tree import DecisionTreeClassifier

# some PCA and GMM stuff
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

import sklearn.ensemble as ske

In [60]:
#train_file = 'https://people.ischool.berkeley.edu/~andrew.morris/data/w209/train.csv'
#test_file = 'https://people.ischool.berkeley.edu/~andrew.morris/data/w209/test.csv'
train_file = 'data/mini_train.csv'
dev_file = 'data/mini_dev.csv'
test_file = 'data/mini_test.csv'

In [61]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', 
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', 
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', 
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', 
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', 
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', 
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', 
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', 
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', 
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

In [62]:
#train_df = pd.read_csv(train_file, nrows=1000, low_memory=False)
#test_df = pd.read_csv(test_file, nrows=1000, low_memory=False)
train_df = pd.read_csv(train_file, dtype=dtypes)
dev_df = pd.read_csv(dev_file, dtype=dtypes)
test_df = pd.read_csv(test_file, dtype=dtypes)

train_df.head()

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,6451622,b92c48db0fd0a9363460c0e2e5ee31dc,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1209.0,0,7.0,0,NaN,...,18154.0,1,NaN,0.0,0,0,0.0,1.0,11.0,1
1,4631013,84ea0624c06444f93088b81223dfd396,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1162.0,0,7.0,0,NaN,...,12199.0,1,NaN,0.0,0,0,0.0,1.0,7.0,0
2,7339290,d292fa249439ce816530dcc1cbf1edac,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1717.0,0,7.0,0,NaN,...,9543.0,0,NaN,0.0,0,0,0.0,0.0,6.0,1
3,7144608,ccfc1d80a0b58a78018756a7abefa6ed,win8defender,1.1.15200.1,4.18.1806.18062,1.275.545.0,0,7.0,0,NaN,...,9011.0,1,0.0,0.0,0,0,0.0,0.0,11.0,1
4,1559551,2cc4018aa15c05a84ac37cd963e9f109,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1257.0,0,7.0,0,NaN,...,63009.0,1,NaN,0.0,0,0,0.0,0.0,1.0,0


In [63]:
train_labels = pd.DataFrame()
train_labels["HasDetections"] = train_df["HasDetections"]
train_data = pd.DataFrame()
train_data = train_df.drop(columns="HasDetections")
mini_train_labels = np.array(train_labels)
mini_train_data = np.array(train_data)

dev_labels = pd.DataFrame()
dev_labels["HasDetections"] = dev_df["HasDetections"]
dev_data = pd.DataFrame()
dev_data = dev_df.drop(columns="HasDetections")
dev_labels = np.array(dev_labels)
dev_data = np.array(dev_data)

test_labels = pd.DataFrame()
test_labels["HasDetections"] = test_df["HasDetections"]
test_data = pd.DataFrame()
test_data = test_df.drop(columns="HasDetections")
test_labels = np.array(test_labels)
test_data = np.array(test_data)

In [66]:
fsel = ske.ExtraTreesClassifier().fit(train_data, train_labels)
model = SelectFromModel(fsel, prefit=True)
train_data_new = model.transform(train_data)
nb_features = train_data_new.shape[1]

/Users/kevin/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: could not convert string to float: 'b92c48db0fd0a9363460c0e2e5ee31dc'

In [56]:
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(train_data.drop(columns="MachineIdentifier"), train_labels)

ValueError: could not convert string to float: 'win8defender'